<a href="https://colab.research.google.com/github/dritx16/transfer-learning/blob/main/mobilenetv2_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #@title Default title text
# from zipfile import ZipFile
# file_name = "/archive.zip"

# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()
#   print('Done')

Done


In [ ]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import PIL
import cv2

import tensorflow as tf
import tensorflow_hub as hub

# print("TF version:", tf.__version__)
# print("Hub version:", hub.__version__)
# print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# **Importing Images**

Dataset link : https://www.kaggle.com/niteshfre/chessman-image-dataset


In [ ]:
import pathlib
path = "/content/Chessman-image-dataset/Chess"    # For other datasets, must be updated.
data_dir = pathlib.Path(path)
# data_dir

In [ ]:
# Class dictionaries
chess_images_dict = {
    'bishop' : list(data_dir.glob('Bishop/*')),
    'king' : list(data_dir.glob('King/*')),
    'knight' : list(data_dir.glob('Knight/*')),
    'pawn' : list(data_dir.glob('Pawn/*')),
    'queen' : list(data_dir.glob('Queen/*')),
    'rook' : list(data_dir.glob('Rook/*')),
}

In [ ]:
chess_labels_dict = {
    'bishop' : 0,
    'king' : 1,
    'knight' : 2,
    'pawn' : 3,
    'queen' : 4,
    'rook' : 5,
}

# **Getting Images into train and test arrays**

In [ ]:
x, y = [], []
IMAGE_SHAPE = (224, 224)    # Required image size for MobileNetV2.
dataset_size = 0
for item_name, items in chess_images_dict.items() :
  for image in items :
    img = cv2.imread(str(image))
    dataset_size += 1
    try :   # In case of errorness file types for cv2.
      resized_img = cv2.resize(img, IMAGE_SHAPE)    # Resizing images to appropriate size for MobileNetV2.
    except :
      continue
    x.append(resized_img)
    y.append(chess_labels_dict[item_name])
# dataset_size

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
# Dividing dataset into train and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75 , random_state=0)

In [ ]:
# Feature scaling
x_train_scaled = x_train / 255
x_test_scaled = x_test / 255

# Transfer Learning Proccess

For this project, MobileNetV2 is used.

In [ ]:
# Importing MobileNetV2 model from tensorflow hub.
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), 
    trainable=False)

In [ ]:
num_of_classes = len(chess_images_dict)   # For other datasets, must be updated.
model = tf.keras.Sequential([
                             pretrained_model_without_top_layer,
                             tf.keras.layers.Dense(num_of_classes)
                             ])
BATCH_SIZE = 16
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 6)                 7686      
                                                                 
Total params: 2,265,670
Trainable params: 7,686
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(x_train_scaled, y_train, epochs=10, steps_per_epoch=len(x_train) // BATCH_SIZE)

Epoch 1/10
26/26 [==============================] - 14s 57ms/step - loss: 1.7770 - accuracy: 0.2957
Epoch 2/10
26/26 [==============================] - 1s 57ms/step - loss: 1.0664 - accuracy: 0.6370
Epoch 3/10
26/26 [==============================] - 1s 57ms/step - loss: 0.7816 - accuracy: 0.7909
Epoch 4/10
26/26 [==============================] - 1s 57ms/step - loss: 0.6196 - accuracy: 0.8534
Epoch 5/10
26/26 [==============================] - 1s 57ms/step - loss: 0.5162 - accuracy: 0.8870
Epoch 6/10
26/26 [==============================] - 1s 57ms/step - loss: 0.4431 - accuracy: 0.9279
Epoch 7/10
26/26 [==============================] - 1s 57ms/step - loss: 0.3706 - accuracy: 0.9471
Epoch 8/10
26/26 [==============================] - 1s 57ms/step - loss: 0.3301 - accuracy: 0.9567
Epoch 9/10
26/26 [==============================] - 2s 58ms/step - loss: 0.2884 - accuracy: 0.9688
Epoch 10/10
26/26 [==============================] - 1s 57ms/step - loss: 0.2513 - accuracy: 0.9808


In [ ]:
model.evaluate(x_test_scaled, y_test)

5/5 [==============================] - 2s 212ms/step - loss: 0.7307 - accuracy: 0.7410


[0.7307054400444031, 0.7410072088241577]